<a href="https://colab.research.google.com/github/rafavidal1709/projeto-aplicado-iii/blob/main/Acur%C3%A1cia_dos_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregando o dataset de avaliação

Vamos conectar ao Google Drive para salvar e carregar arquivos permanentes.

Também vamos criar funções para lidar com o carregamento, armazenamento e transformação do dataset em seu formato original.

In [1]:
from google.colab import drive
import os
import json
from google.colab import files

drive.mount('/content/drive')

# Caminho para a pasta no Google Drive
folder_path = '/content/drive/MyDrive/projeto-aplicado-iii'

# Verificar se a pasta existe, se não, criar
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Faz o upload de um dataset
def upload_dataset():
  uploaded = files.upload()
  for file_name in uploaded.keys():
      with open(file_name, 'r', encoding='utf-8') as f:
          data = json.load(f)
  return data, file_name

# Formata o dataset original para o formato desejado
def format_dataset(data):
  dataset = {'text':[],'category':[],'embedding':{},'accuracy':{}}
  for c in range(len(data)):
    for i in data[c]['examples']:
      dataset['text'].append(i)
      dataset['category'].append(data[c]['category'])
  return dataset

# Salva o dataset no Google Drive
def save_dataset(dataset, filename):
  file_path = os.path.join(folder_path, filename+'.json')
  with open(file_path, 'w') as f:
    json.dump(dataset, f)

# Executa a sequência das três funções anteriores
def full_upload_dataset():
  data, file_name = upload_dataset()
  dataset = format_dataset(data)
  save_dataset(dataset,file_name)
  return dataset

# Carrega o dataset do Google Drive
def load_dataset(filename):
  file_path = os.path.join(folder_path, filename+'.json')
  with open(file_path, 'r') as f:
    data = json.load(f)
  data['embedding'] = {int(k): v for k, v in data['embedding'].items()}
  return data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Se o dataset ainda não foi baixado e salvo no Google Drive:

In [ ]:
dataset = full_upload_dataset()

Saving dataset_accuracy_2.json to dataset_accuracy_2.json


OU se ele já está no formato correto no Google Drive:

In [83]:
dataset = load_dataset("dataset_accuracy")
dataset2 = load_dataset("dataset_accuracy_2")

Também pode salvá-lo no Google Drive:

In [110]:
#save_dataset(dataset, "dataset_accuracy")
#save_dataset(dataset2, "dataset_accuracy_2")

# Embedding com Longformer

In [68]:
import torch
from transformers import LongformerTokenizer, LongformerModel

class LongformerEmbedding:
  def __init__(self):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Verificar se a GPU está disponível
    torch.cuda.empty_cache()  # Liberar memória da GPU, se necessário
    self.tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-large-4096") # Carregar o modelo e o tokenizador Longformer-large
    self.model = LongformerModel.from_pretrained("allenai/longformer-large-4096").to(self.device)
    self.model.gradient_checkpointing_enable()  # Habilitar gradient checkpointing para economizar memória

  def process_text(self, text):
    with torch.no_grad():  # Desabilitar o cálculo de gradientes
        # Reduzir o comprimento máximo, se possível
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=2048)

        # Definir a máscara de atenção: 1 para global attention no primeiro token (ou outro token especial)
        attention_mask = torch.ones(inputs['input_ids'].shape, dtype=torch.long).to(self.device)  # Enviar para GPU
        global_attention_mask = torch.zeros(inputs['input_ids'].shape, dtype=torch.long).to(self.device)  # Enviar para GPU
        global_attention_mask[:, 0] = 1  # Dar atenção global ao primeiro token (posição 0)

        # Enviar os inputs para a GPU
        inputs = {key: value.to(self.device) for key, value in inputs.items()}

        # Passar pelo modelo com a máscara de atenção global
        outputs = self.model(input_ids=inputs['input_ids'], attention_mask=attention_mask, global_attention_mask=global_attention_mask)

        # Extraímos o embedding do primeiro token (com atenção global)
        global_attention_embedding = outputs.last_hidden_state[:, 0, :]  # Primeiro token com global attention

        # Fazer a média dos embeddings de todos os tokens (global pooling)
        output_mean_embedding = torch.mean(outputs.last_hidden_state, dim=1)

        # Mover os embeddings para a CPU
        global_attention_embedding = global_attention_embedding.cpu()
        output_mean_embedding = output_mean_embedding.cpu()

        # Sincronizar CUDA para garantir que a GPU terminou o processamento antes de prosseguir
        torch.cuda.synchronize()

        # Limpar variáveis não utilizadas e liberar memória da GPU
        del inputs, attention_mask, global_attention_mask, outputs
        torch.cuda.empty_cache()

        return {"longformer_global_attention": global_attention_embedding, "longformer_output_mean": output_mean_embedding}

longformer_embedding = LongformerEmbedding()
for i in range(len(dataset['text'])):
  embedding = longformer_embedding.process_text(dataset['text'][i])
  if i not in dataset['embedding']:
        dataset['embedding'][i] = {}
  for key, value in embedding.items():
    dataset['embedding'][i][key] = value.numpy().tolist()[0]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Embedding com BERT

In [70]:
import torch
from transformers import BertTokenizer, BertModel

class BertEmbedding:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Verificar se a GPU está disponível
        torch.cuda.empty_cache()  # Liberar memória da GPU, se necessário
        self.tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")  # Carregar o modelo e o tokenizador BERT-large
        self.model = BertModel.from_pretrained("bert-large-uncased").to(self.device)

    def process_text(self, text):
        with torch.no_grad():  # Desabilitar o cálculo de gradientes
            # Reduzir o comprimento máximo, se possível
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

            # Enviar os inputs para a GPU
            inputs = {key: value.to(self.device) for key, value in inputs.items()}

            # Passar pelo modelo
            outputs = self.model(**inputs)

            # Pooler output: já é o resumo global baseado no token [CLS]
            pooler_output = outputs.pooler_output.cpu()  # Já é um resumo do texto

            # Fazer a média dos embeddings de todos os tokens (global pooling)
            output_mean_embedding = torch.mean(outputs.last_hidden_state, dim=1)
            output_mean_embedding = output_mean_embedding.cpu()

            # Sincronizar CUDA para garantir que a GPU terminou o processamento antes de prosseguir
            torch.cuda.synchronize()

            # Limpar variáveis não utilizadas e liberar memória da GPU
            del inputs, outputs
            torch.cuda.empty_cache()

            return {"bert_pooler_output": pooler_output, "bert_output_mean": output_mean_embedding}

bert_embedding = BertEmbedding()
for i in range(len(dataset['text'])):
    embedding = bert_embedding.process_text(dataset['text'][i])
    if i not in dataset['embedding']:
        dataset['embedding'][i] = {}
    for key, value in embedding.items():
        dataset['embedding'][i][key] = value.numpy().tolist()[0]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Embedding com BERTimbau

In [77]:
import torch
from transformers import BertTokenizer, BertModel

class BertimbauEmbedding:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Verificar se a GPU está disponível
        torch.cuda.empty_cache()  # Liberar memória da GPU, se necessário
        self.tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-large-portuguese-cased")  # Carregar o modelo e o tokenizador BERTimbau-large
        self.model = BertModel.from_pretrained("neuralmind/bert-large-portuguese-cased").to(self.device)

    def process_text(self, text):
        with torch.no_grad():  # Desabilitar o cálculo de gradientes
            # Reduzir o comprimento máximo, se possível
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

            # Enviar os inputs para a GPU
            inputs = {key: value.to(self.device) for key, value in inputs.items()}

            # Passar pelo modelo
            outputs = self.model(**inputs)

            # Pooler output: já é o resumo global baseado no token [CLS]
            pooler_output = outputs.pooler_output.cpu()  # Já é um resumo do texto

            # Fazer a média dos embeddings de todos os tokens (global pooling)
            output_mean_embedding = torch.mean(outputs.last_hidden_state, dim=1)
            output_mean_embedding = output_mean_embedding.cpu()

            # Sincronizar CUDA para garantir que a GPU terminou o processamento antes de prosseguir
            torch.cuda.synchronize()

            # Limpar variáveis não utilizadas e liberar memória da GPU
            del inputs, outputs
            torch.cuda.empty_cache()

            return {"bertimbau_pooler_output": pooler_output, "bertimbau_output_mean": output_mean_embedding}

bertimbau_embedding = BertimbauEmbedding()
for i in range(len(dataset['text'])):
    embedding = bertimbau_embedding.process_text(dataset['text'][i])
    if i not in dataset['embedding']:
        dataset['embedding'][i] = {}
    for key, value in embedding.items():
        dataset['embedding'][i][key] = value.numpy().tolist()[0]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Métrica de acurácia

In [72]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def accuracy_test_cosine(dataset, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    for i in range(len(dataset['text'])):
        # Calculando similaridade cosseno entre o embedding atual e todos os outros embeddings
        similarities = cosine_similarity([embeddings[i]], embeddings)
        ranked_indices = np.argsort(similarities[0])[::-1]  # Ordenar por similaridade

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):
            if dataset['category'][i] == dataset['category'][ranked_indices[j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

In [73]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def accuracy_test_knn(dataset, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    # Convertendo a lista de embeddings para um array do numpy
    embeddings = np.array(embeddings)

    # Usando NearestNeighbors para calcular os k vizinhos mais próximos (k=top_n)
    knn = NearestNeighbors(n_neighbors=len(dataset['text']), metric='euclidean')
    knn.fit(embeddings)

    for i in range(len(dataset['text'])):
        # Encontrando os k vizinhos mais próximos para o embedding atual
        distances, ranked_indices = knn.kneighbors([embeddings[i]])

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):  # Considerando apenas os top_n vizinhos
            if dataset['category'][i] == dataset['category'][ranked_indices[0][j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

# Avaliação dos modelos



In [96]:
dataset['accuracy']['cosine_similarity']={}
for emb in dataset['embedding'][0].keys():
  for key, value in accuracy_test_cosine(dataset, emb).items():
    dataset['accuracy']['cosine_similarity'][key] = value

display(dataset['accuracy']['cosine_similarity'])

{'longformer_global_attention': {'mean': 107.6,
  'mean_per_cat': {'trabalhadores_sem_epi': 86.8,
   'contaminacao_nascente_agrotoxico': 122.65,
   'pisoteamento_nascente': 119.15,
   'agrotoxico_proximo_residencias': 115.7,
   'queimadas': 93.7},
  'matrix': [[0.0, 4.0, 9.0, 16.0, 17.0, 19.0, 21.0, 27.0],
   [0.0, 3.0, 5.0, 12.0, 19.0, 21.0, 25.0, 28.0],
   [0.0, 8.0, 12.0, 20.0, 21.0, 26.0, 31.0, 33.0],
   [0.0, 2.0, 20.0, 29.0, 32.0, 36.0, 42.0, 45.0],
   [0.0, 10.0, 13.0, 17.0, 25.0, 33.0, 41.0, 45.0],
   [0.0, 6.0, 8.0, 11.0, 13.0, 20.0, 27.0, 35.0],
   [0.0, 6.0, 8.0, 19.0, 23.0, 28.0, 31.0, 32.0],
   [0.0, 7.0, 13.0, 17.0, 28.0, 29.0, 32.0, 35.0],
   [0.0, 4.0, 7.0, 16.0, 17.0, 21.0, 27.0, 32.0],
   [0.0, 2.0, 15.0, 27.0, 29.0, 30.0, 31.0, 45.0],
   [0.0, 8.0, 9.0, 12.0, 20.0, 22.0, 35.0, 40.0],
   [0.0, 12.0, 14.0, 28.0, 30.0, 31.0, 38.0, 39.0],
   [0.0, 2.0, 12.0, 13.0, 17.0, 22.0, 41.0, 46.0],
   [0.0, 6.0, 14.0, 15.0, 20.0, 26.0, 31.0, 32.0],
   [0.0, 6.0, 7.0, 8.0, 16.0, 19

In [97]:
dataset['accuracy']['knn_euclidian']={}
for emb in dataset['embedding'][0].keys():
  for key, value in accuracy_test_knn(dataset, emb).items():
    dataset['accuracy']['knn_euclidian'][key] = value

display(dataset['accuracy']['knn_euclidian'])

{'longformer_global_attention': {'mean': 107.6,
  'mean_per_cat': {'trabalhadores_sem_epi': 86.8,
   'contaminacao_nascente_agrotoxico': 122.65,
   'pisoteamento_nascente': 119.15,
   'agrotoxico_proximo_residencias': 115.7,
   'queimadas': 93.7},
  'matrix': [[0.0, 4.0, 9.0, 16.0, 17.0, 19.0, 21.0, 27.0],
   [0.0, 3.0, 5.0, 12.0, 19.0, 21.0, 25.0, 28.0],
   [0.0, 8.0, 12.0, 20.0, 21.0, 26.0, 31.0, 33.0],
   [0.0, 2.0, 20.0, 29.0, 32.0, 36.0, 42.0, 45.0],
   [0.0, 10.0, 13.0, 17.0, 25.0, 33.0, 41.0, 45.0],
   [0.0, 6.0, 8.0, 11.0, 13.0, 20.0, 27.0, 35.0],
   [0.0, 6.0, 8.0, 19.0, 23.0, 28.0, 31.0, 32.0],
   [0.0, 7.0, 13.0, 17.0, 28.0, 29.0, 32.0, 35.0],
   [0.0, 4.0, 7.0, 16.0, 17.0, 21.0, 27.0, 32.0],
   [0.0, 2.0, 15.0, 27.0, 29.0, 30.0, 31.0, 45.0],
   [0.0, 8.0, 9.0, 12.0, 20.0, 22.0, 35.0, 40.0],
   [0.0, 12.0, 14.0, 28.0, 30.0, 31.0, 38.0, 39.0],
   [0.0, 2.0, 12.0, 13.0, 17.0, 22.0, 41.0, 46.0],
   [0.0, 6.0, 14.0, 15.0, 20.0, 26.0, 31.0, 32.0],
   [0.0, 6.0, 7.0, 8.0, 16.0, 19

# Métrica de acurácia 2

In [99]:
dataset['embedding'][0].keys()

dict_keys(['longformer_global_attention', 'longformer_output_mean', 'bert_pooler_output', 'bert_output_mean', 'bertimbau_pooler_output', 'bertimbau_output_mean'])

In [100]:
dataset2['embedding'][0].keys()

dict_keys(['longformer_global_attention', 'longformer_output_mean', 'bert_pooler_output', 'bert_output_mean', 'bertimbau_pooler_output', 'bertimbau_output_mean'])

# Avaliação dos modelos 2

In [103]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def accuracy2_test_cosine(dataset, dataset2, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset2['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    embeddings2 = []
    for i in range(len(dataset2['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings2.append(dataset2['embedding'][i][embedding])

    for i in range(len(dataset2['text'])):
        # Calculando similaridade cosseno entre o embedding atual e todos os outros embeddings
        similarities = cosine_similarity([embeddings2[i]], embeddings)
        ranked_indices = np.argsort(similarities[0])[::-1]  # Ordenar por similaridade

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):
            if dataset2['category'][i] == dataset['category'][ranked_indices[j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset2['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset2['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

In [108]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def accuracy2_test_knn(dataset, dataset2, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset2['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    # Convertendo a lista de embeddings para um array do numpy
    embeddings = np.array(embeddings)

    embeddings2 = []
    for i in range(len(dataset2['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings2.append(dataset2['embedding'][i][embedding])

    # Convertendo a lista de embeddings para um array do numpy
    embeddings2 = np.array(embeddings2)

    # Usando NearestNeighbors para calcular os k vizinhos mais próximos (k=top_n)
    knn = NearestNeighbors(n_neighbors=len(dataset['text']), metric='euclidean')
    knn.fit(embeddings)

    for i in range(len(dataset2['text'])):
        # Encontrando os k vizinhos mais próximos para o embedding atual
        distances, ranked_indices = knn.kneighbors([embeddings2[i]])

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):  # Considerando apenas os top_n vizinhos
            if dataset['category'][i] == dataset['category'][ranked_indices[0][j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset2['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset2['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

In [104]:
dataset2['accuracy']['cosine_similarity']={}
for emb in dataset2['embedding'][0].keys():
  for key, value in accuracy2_test_cosine(dataset, dataset2, emb).items():
    dataset2['accuracy']['cosine_similarity'][key] = value

display(dataset2['accuracy']['cosine_similarity'])

{'cosine_similarity': {'longformer_global_attention': {'mean': 140.4,
   'mean_per_cat': {'trabalhadores_sem_epi': 161.4,
    'contaminacao_nascente_agrotoxico': 153.1,
    'pisoteamento_nascente': 133.2,
    'agrotoxico_proximo_residencias': 143.7,
    'queimadas': 110.6},
   'matrix': [[6.0, 7.0, 19.0, 25.0, 34.0, 38.0, 40.0, 42.0],
    [2.0, 5.0, 9.0, 23.0, 26.0, 27.0, 37.0, 40.0],
    [1.0, 16.0, 17.0, 18.0, 23.0, 25.0, 29.0, 34.0],
    [10.0, 13.0, 15.0, 16.0, 31.0, 34.0, 35.0, 39.0],
    [0.0, 7.0, 11.0, 24.0, 28.0, 32.0, 36.0, 44.0],
    [3.0, 4.0, 13.0, 16.0, 21.0, 22.0, 23.0, 32.0],
    [0.0, 6.0, 7.0, 14.0, 20.0, 24.0, 26.0, 27.0],
    [4.0, 6.0, 7.0, 25.0, 29.0, 30.0, 39.0, 40.0],
    [7.0, 14.0, 16.0, 21.0, 22.0, 35.0, 36.0, 39.0],
    [12.0, 16.0, 17.0, 18.0, 20.0, 24.0, 27.0, 37.0],
    [9.0, 10.0, 11.0, 23.0, 28.0, 39.0, 42.0, 43.0],
    [1.0, 3.0, 11.0, 18.0, 37.0, 39.0, 40.0, 44.0],
    [1.0, 6.0, 12.0, 18.0, 35.0, 38.0, 43.0, 45.0],
    [4.0, 9.0, 10.0, 14.0, 17.0, 19

In [109]:
dataset2['accuracy']['knn_euclidian']={}
for emb in dataset2['embedding'][0].keys():
  for key, value in accuracy2_test_knn(dataset, dataset2, emb).items():
    dataset2['accuracy']['knn_euclidian'][key] = value

display(dataset2['accuracy']['knn_euclidian'])

{'longformer_global_attention': {'mean': 144.44,
  'mean_per_cat': {'trabalhadores_sem_epi': 136.8,
   'contaminacao_nascente_agrotoxico': 142.8,
   'pisoteamento_nascente': 141.8,
   'agrotoxico_proximo_residencias': 143.7,
   'queimadas': 157.1},
  'matrix': [[6.0, 7.0, 19.0, 25.0, 34.0, 38.0, 40.0, 42.0],
   [2.0, 5.0, 9.0, 23.0, 26.0, 27.0, 37.0, 40.0],
   [1.0, 16.0, 17.0, 18.0, 23.0, 25.0, 29.0, 34.0],
   [10.0, 13.0, 15.0, 16.0, 31.0, 34.0, 35.0, 39.0],
   [0.0, 7.0, 11.0, 24.0, 28.0, 32.0, 36.0, 44.0],
   [3.0, 4.0, 13.0, 16.0, 21.0, 22.0, 23.0, 32.0],
   [0.0, 6.0, 7.0, 14.0, 20.0, 24.0, 26.0, 27.0],
   [4.0, 6.0, 7.0, 25.0, 29.0, 30.0, 39.0, 40.0],
   [7.0, 14.0, 16.0, 21.0, 22.0, 35.0, 36.0, 39.0],
   [12.0, 16.0, 17.0, 18.0, 20.0, 24.0, 27.0, 37.0],
   [4.0, 7.0, 8.0, 25.0, 27.0, 31.0, 35.0, 40.0],
   [0.0, 8.0, 13.0, 14.0, 29.0, 33.0, 41.0, 43.0],
   [0.0, 8.0, 15.0, 16.0, 23.0, 28.0, 41.0, 42.0],
   [0.0, 6.0, 13.0, 16.0, 27.0, 29.0, 33.0, 36.0],
   [4.0, 6.0, 8.0, 18.0, 